**извлечение признаков из текста на естественном языке**

класстеризатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
1880,"Весной нынешнего года в Париже был сделано уникальное открытие - обнаружен рисунок Леонардо да Винчи.\n\nРисунок да Винчи принесли на аукцион случайно\n\nКак сообщают СМИ, французский доктор на пе...",culture
1415,"МИД Японии официально объявил программу визита в Японию президента России Владимира Путина, сообщает ТАСС. Ожидается, что 15 декабря президент прибудет в аэропорт префектуры Ямагути на юго-западе ...",politics


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
stopwords = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 
    'так', 'его', 'но',  'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или',  'ни', 'быть', 'был', 'него', 'до', 
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 
    'ж', 'тогда', 'кто','этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
    'этом', 'один', 'почти', 'мой', 'тем','чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем',
    'всех', 'никогда', 'можно', 'при', 'наконец', 'два','об', 'другой', 'хоть', 'после', 
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них','какая', 'много', 
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 
    'всегда', 'конечно', 'всю', 'между']

In [8]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in stopwords ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF-IDF

In [9]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(preprocessor=preprocessor)
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f5cf8a86d08>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [10]:
# размер словаря
len(tf.vocabulary_)

37614

## формируем датасеты

In [11]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37614)

## кластеризируем

In [12]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [13]:
%%time

n_clusters = len(set(data['tag']))
clust = KMeans(n_clusters=n_clusters, random_state=get_seed() )
clust.fit(X)

CPU times: user 51.7 s, sys: 75.6 ms, total: 51.8 s
Wall time: 43.6 s


In [14]:
# set(clust.labels_)

In [15]:
data['cluster'] = clust.labels_

## тестируем

In [16]:
data[ data['cluster']==0 ].sample(10)

,text,tag,cluster
2181,"На трассе «Ленинск-Кузнецкий - Новокузнецк» во время работы на месте массового ДТП пострадали два сотрудника МЧС и трое работников скорой помощи.\n\nКак рассказали в региональном управлении МВД, и...",incident,0
2318,"Сотрудница полиции из подмосковного Серпухова в нерабочее время на собственном автомобиле насмерть сбила двух пешеходов. Об этом сообщила пресс-служба регионального управления МВД.\n\nОтмечается, ...",incident,0
2250,"СК проверяет данные о том, что в Перми ребенка высадили из автобуса в лесу\n\nМОСКВА, 13 дек — РИА Новости. Пермские следователи проводят доследственную проверку сообщений СМИ о том, что водитель ...",incident,0
2275,"Пользователи Сети опубликовали видео с места массовой аварии на трассе в Кемеровской области. На кадрах видно, что на месте происшествия работают отряды спасателей.\n\nМногие автомобили оказались ...",incident,0
1270,Пассажирке пришлось сесть за руль автобуса Ecolines Таллин — Рига вместо пьяного водителя\n\n27 ноября 2016 в 16:00\n\nAUTO.TUT.BY\n\nМолодой эстонке Трийн Кибур пришлось взять на себя управление ...,auto,0
587,"В Смолевичском районе Renault вылетел в кювет и врезался в дерево: водитель погиб\n\n30 ноября 2016 в 10:36\n\nAUTO.TUT.BY\n\nАвария произошла 29 ноября, около 00.30, вблизи деревни Потичево Смоле...",incident,0
2323,"Сотрудница межмуниципального управления МВД России «Серпуховское», находившаяся за рулем в пьяном виде, насмерть сбила двоих человек в Подмосковье.\n\nПо информации главного управления МВД по Моск...",incident,0
2179,"13.12.16 09:53\n\nКемерово. 13 декабря. ИНТЕРФАКС-СИБИРЬ - Бригада ""скорой помощи"", два спасателя и двое автомобилистов госпитализированы с травмами различной степени тяжести после ДТП на трассе ""...",incident,0
2382,"По данным пресс-службы столичной автоинспекции, авария произошла на пересечение улицы Мосфильмовская с Университетским проспектом. Девочек, которым 8 и 9 лет, направили в больницу на осмотр. По пр...",incident,0
510,В Бресте пьяный водитель на Mercedes по дороге в магазин за добавкой врезался в столб\n\n9 декабря 2016 в 15:41\n\nTUT.BY\n\n23-летний житель Бреста врезался в столб по дороге в магазин.\n\nФото: ...,incident,0


In [17]:
data[ data['cluster']==1 ].sample(10)

,text,tag,cluster
951,ЕБРР может начать предоставлять кредиты по программе разделения рисков в белорусских рублях\n\n7 декабря 2016 в 17:40\n\nБЕЛТА\n\nЕвропейский банк реконструкции и развития (ЕБРР) может начать пред...,economics,1
2444,Теги: Общество суд цирк\n\nДрессировщик Эдгард Запашный через суд требует с актера Валерия Николаева компенсацию морального вреда за высказывания о жестоком обращении с животными в Московском цирк...,incident,1
1766,"06:20\n\nНовый член совета директоров Банка России Ольга Полякова, которая возглавляет главное управление по Центральному федеральному округу (ЦФО), будет назначена зампредом ЦБ. Об этом пишет «Ко...",economics,1
1337,"Эксперт: ""Адмирал Кузнецов"" способен уничтожить авианосец с прикрытием одним залпом\n\n2 декабря 2016 в 16:42\n\n42.TUT.BY\n\nРазработчик оружия для тяжелого авианесущего крейсера Северного флота ...",tech,1
525,В Пружанском районе медсестра психиатрической больницы подозревается в краже пенсий у пациентов\n\n7 декабря 2016 в 13:04\n\nTUT.BY\n\nСледственный комитет возбудил уголовное дело в отношении 52-л...,incident,1
3085,Информационный портал Республики Коми (komipress.ru) (Сыктывкар) Активисты\nкоми народа обеспокоены ?похоронами? языка коренного этноса республики\nАктивисты коми народа выразили озабоченность выс...,social,1
1305,"""Великий белорусский файервол"": как и зачем у нас блокируют анонимный доступ к интернету\n\n6 декабря 2016 в 18:56\n\n42.TUT.BY\n\nВ начале декабря в Беларуси начали блокировку серверов Tor. Тепер...",tech,1
1356,"Стало известно, какие ракеты использует на учениях Украина\n\n1 декабря 2016 в 11:49\n\nДенис Алдохин / 42.TUT.BY\n\nВ учебных ракетных стрельбах на юге Украины участвует ЗРС С-300. Об этом телека...",politics,1
262,"В Волгограде появится музей сталинской эпохи Как сообщила агентству\n""Интерфакс-Юг"" заместитель директора музея- панорамы ""Сталинградская\nбитва"" Светлана Аргасцева, музей будет размещен в культур...",culture,1
167,"Хорватия возглавила рейтинг Кубка Дэвиса В понедельник был опубликован\nновый рейтинг Кубка Дэвиса, который возглавили триумфаторы розыгрыша-2005\nтеннисисты из Хорватии.Они потеснили на вторую ст...",sport,1


In [18]:
data[ data['cluster']==2 ].sample(10)

,text,tag,cluster
2457,"Персонаж комиксов DC Comics ""Чудо-женщина"" (Wonder Woman) с пятницы, 16 декабря, больше не будет почетным послом ООН по борьбе за права и возможности женщин и девочек, сообщают британская газета T...",culture,2
2236,"\n\n\n\nФото: иллюстрация\n\nПо факту трагического инцидента, произошедшего на соревнованиях по баскетболу в Красноярской средней школе, в результате которого 16-летний школьник скончался, следств...",incident,2
2232,"Причиной смерти мальчика называют врожденное заболевание сердца, однако Следственный комитет предполагают, что соревнования были организованы недостаточно хорошо – в школе не дежурила скорая помощ...",incident,2
2569,"Потенциальной угрозой всему миру учёные называют Коронавирус Ближневосточного респираторного синдрома также Эболу, и некоторые вирусы, которые переносят насекомые.На сегодняшний день распространен...",science,2
2662,"Внеземная жизнь может существовать в верхних слоях атмосферы коричневых карликов, считают ученые из Университета Эдинбурга (Великобритания). Об этом пишет издание Science.\n\n\n\nКоричневыми карли...",science,2
2568,"Британские учёные из Эдинбургского университета назвали 37 возможных пандемий, которые способны возникнуть по всему миру. Каждый из этих вирусов способен активно распространяться между людьми.\n\n...",science,2
2431,"""Яндекс.Музыка"": в Казани живет главный поклонник группы Iowa в России\n\nСтало известно о том, какую музыку слушают жители Татарстана в Интернете. «Яндекс.Музыка» обнародовала рейтинг самых попул...",culture,2
2656,"Ученые назвали планеты, где возможна внеземная жизнь. По данным астрономов из эдинбургского университета, жизнь может существовать в атмосферных слоях коричневых карликов или, как их называют, неу...",science,2
2639,"Ученые выяснили сколько нужно спать, чтобы быть бодрым весь день\n\nИсследователи из Финляндии определили оптимальное количество времени для сна. Они провели ряд экспериментов, в которых приняли у...",science,2
2609,"\r\nСпециалисты из американского университета биологических исследований выяснили, как мытье головы влияет на преждевременное нарушение мозгового кровообращения. Как сообщают ученые, одной из глав...",science,2


In [19]:
data[ data['cluster']==3 ].sample(10)

,text,tag,cluster
2947,Популярный кроссовер Audi Q3 новой генерации пройдет процедуру даунсайзинга.\r\n\r\nАвтомобиль сменит платформу и обзаведется компактными моторами.\r\n\r\nПроизводитель намерен уменьшить размеры д...,auto,3
575,Разбитые окна и покореженная крыша — МАЗ провел краш-тест новой модели школьного автобуса\r\n\r\n1 декабря 2016 в 14:58\r\n\r\nAUTO.TUT.BY\r\n\r\nРазработчики подвергли новую модель школьного авто...,auto,3
2921,"Нашумевший внедорожник Beijing BJ80 производства китайского автоконцерна BAIC, который приобрёл известность, как «клон» Гелендвагена Mercedes-Benz G-Class, в ближайшее время получит спортивную мод...",auto,3
2984,"Японская компания Mitsubishi вывела на дорожные тесты свой новый компактвэн повышенной проходимости. Предсерийный прототип модели, предназначенной для рынка Индонезии, запечатлели автомобильные фо...",auto,3
2919,"\r\nКитайский производитель Beijing создал новый внедорожник, который на предстоящей автомобильной выставке представят, как BJ80. Указанному транспортному средству изготовили кузов в угловатой фор...",auto,3
2924,"Новая версия китайского рамного внедорожника Beijing BJ80, копирующего внешность Mercedes-Benz G-Class, засветилась на свежих фото.\n\nПо предварительной информации, новый вариант китайского внедо...",auto,3
2866,"Дешевле всего внедорожник SsangYong Tivoli будет стоить в виде ""короткой"" переднеприводной машины в базовой комплектации Welcome, оснащенной механической коробкой передач, АБС и кондиционером. Цен...",auto,3
2863,"Компания Sollers, уже много лет представляющая в России марку SsangYong, планировала возобновить продажи этих корейских машин еще в ноябре. Новой ударной силой должен стать компактный кроссовер Ti...",auto,3
2713,"МОСКВА, 13 дек — РИА Новости. Минпромторг выдал 150 миллионов рублей субсидии российскому разработчику ""Т-Платформы"" на создание сверхпрочного ноутбука. Предполагается, что устройство будет защище...",tech,3
2988,"19:26 - 12.12.2016\n\nВо время тестов автомобиль был покрыт камуфляжной пленкой.\n\nФотографии минивэна, получившего увеличенный клиренс, опубликовало издание AutonetMagz.\n\nКонцепт-кар XM был пр...",auto,3


In [20]:
data[ data['cluster']==4 ].sample(10)

,text,tag,cluster
1535,"Схему хищения бюджетных средств, выделенных на реконструкцию ФГБУ «Федеральный медицинский исследовательский центр психиатрии и наркологии им. В.П.Сербского» раскрыли оперативники при содействии М...",incident,4
1641,"На Камчатке может произойти взрывное извержение вулкана Безымянный, одного из наиболее активных в мире. Об этом рассказала руководитель Камчатской группы реагирования на вулканические извержения И...",science,4
3113,ИА Татар-информ Патриарх Кирилл посетит Камчатку 20-23 сентября Патриарх\nМосковский и всея Руси Кирилл посетит Камчатку. Вопросы подготовки к\nпоездке обсуждались на встрече патриарха с губернат...,social,4
3142,"Комиинформ (komiinform.ru) (Сыктывкар) Главы регионов будут устанавливать\nобъемы добычи охотничьих ресурсов для представителей малочисленных\nнародов Законопроект, определяющий особенности охоты,...",politics,4
1642,Сильное извержение вулкана Безымянный может произойти в ближайшее время на Камчатке. Об этом заявила руководитель Камчатской группы реагирования на вулканические извержения (KVERT) Института вулка...,science,4
1542,"""Сотрудниками Главного управления экономической безопасности и противодействия коррупции МВД России совместно с ГУ МВД России по г. Москве, при содействии Минздрава России выявлена схема хищения б...",incident,4
3135,"Север ДВ (severdv.ru) В селе Гадля Ольского района работает детский\nэтнокультурный лагерь ""Нёлтэн Хедекэн"" В лагере завершается трехгодичный\nэксперимент по погружению детей в языковую, культурну...",culture,4
3168,"Законодательная дума Хабаровского края (duma.khv.ru) Состоялось\nочередное заседание Думы На последнем перед каникулами очередном заседании\nЗаконодательной Думы Хабаровского края, состоявшемся 28...",politics,4
3190,"ИА ""Время города"" Губернатора просят выделить еще 40 млн рублей на соцподдержку\nжителей Эвенкии Члены комитета Заксобрания Красноярского края по делам Севера\nи КМНС обратились к губернатору края...",social,4
3167,ИА Сахалин-Курилы 25 мастеров представили свои работы на первом областном\nфестивале ремесел коренных народов Сахалина Первый областной фестиваль ремесел\nкоренных малочисленных народов Севера Сах...,culture,4


In [21]:
data[ data['cluster']==9 ].sample(10)

,text,tag,cluster
2397,"Объявлен полный список номинантов на вторую после ""Оскара"" по значимости кинопремию ""Золотой глобус"". В этот раз не обошлось без приятных неожиданностей - целых двух. Неприличный и разудалый кинок...",culture,9
2414,"Ассоциация кинокритиков США и Канады назвала лучших актёра и актрису года — ими стали Кейси Аффлек за главную роль в фильме ""Манчестер у моря"" и Натали Портман, сыгравшая в картине ""Джеки"".\n\nПре...",culture,9
205,"В Лос-Анджелесе объявлены номинации на премию ""Энни"" Номинации на\nамериканскую премию ""Энни"", присуждаемую за лучшие работы в области\nмультипликации, объявлены в Лос-Анджелесе. Безусловным лидер...",culture,9
2420,"В Санта-Монике (Калифорния) прошла 22-я церемония вручения наград ассоциации кинокритиков США и Канады Critics’ Choice Awards.\n\nЛучшим фильмом признан мюзикл ""Ла-Ла Ленд"" с Райаном Гослингом и Э...",culture,9
2419,"Американский фильм-мюзикл «Ла-Ла Ленд» получил премию Critics Choice Award в номинации «Лучший фильм». Кинокартину снял Демиен Шазелл, который был удостоен награды как лучший режиссер. Церемония в...",culture,9
870,"Трейлеры недели: ""Мумия"" с Томом Крузом, Бондарчук в роли Деда Мороза и нестареющий Джеки Чан\n\n5 декабря 2016 в 9:25\n\nАртем Бордовский, AFISHA.TUT.BY\n\nПосле нескольких недель с сочными подбо...",culture,9
2422,"ВАШИНГТОН, 12 дек – РИА Новости. Мюзикл ""Ла-ла-ленд"" (La La Land) Дэмьена Шазелла ожидаемо взял главную награду премии Critics Choice, 22-я церемония вручения награды ассоциации кинокритиков США и...",culture,9
2489,В голосовании приняли участие более 3 тысяч кинодеятелей из всех стран Европы\n\nВчера вечером в культурной столице Европы — польском Вроцлаве — прошла ежегодная церемония награждения European Fil...,culture,9
2409,"ВАШИНГТОН, 12 дек – РИА Новости. Песней, танцами и легкостью может запомниться 2016 киногод: мюзикл ""Ла-ла-ленд"" ожидаемо получил большинство номинаций на премию Ассоциации зарубежных журналистов ...",culture,9
2493,"Пирса Броснана накормили попкорном, потом наградили европейским «Оскаром»\n\nРоссийский композитор Илья Демуцкий получил награду вне конкурса\n\n10 декабря в польском Вроцлаве, который стал в 2016...",culture,9
